In [ ]:
a = df.corr()
fig, ax = plt.subplots(figsize = (10,10))
#二維的陣列的熱力圖，橫軸和數軸的ticklabels要加上去的話，既可以通過將array轉換成有column
#和index的DataFrame直接繪圖生成，也可以後續再加上去。後面加上去的話，更靈活，包括可設定labels大小方向等。
sns.heatmap(pd.DataFrame(np.round(a,2), columns =df.columns, index = df.columns), 
                annot=True, vmax=1,vmin = 0, xticklabels= True, yticklabels= True, square=True, cmap="YlGnBu")
#sns.heatmap(np.round(a,2), annot=True, vmax=1,vmin = 0, xticklabels= True, yticklabels= True, 
#            square=True, cmap="YlGnBu")
ax.set_title('二維陣列熱力圖', fontsize = 18)
ax.set_ylabel('特徵', fontsize = 18)
ax.set_xlabel('特徵', fontsize = 18) #橫變成y軸，跟矩陣原始的佈局情況是一樣的

In [ ]:
df2=df['Air Pressure']


df2.plot(kind='kde',title='Air Pressure')
plt.show()

print('標準差 :',df2.std())

In [ ]:
df2=df['Air Temperature']


df2.plot(kind='kde',title='	Air Temperature')
plt.show()

print('標準差 :',df2.std())

In [ ]:
df2=df['Wind Speed']


df2.plot(kind='kde',title='Wind Speed')
plt.show()

print('標準差 :',df2.std())

In [ ]:
df2=df['Precipitation']


df2.plot(kind='kde',title='Precipitation')
plt.show()

print('標準差 :',df2.std())

In [ ]:
df2=df['PM2.5']


df2.plot(kind='kde',title='PM2.5')
plt.show()

print('標準差 :',df2.std())

In [ ]:
# plt.boxplot(y,showfliers=True)
# plt.show()

r=plt.boxplot(y,showfliers=True)

print('異常値: ',r['fliers'][0].get_data()[1])#取得異常値

In [ ]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(max_depth=5, random_state=0,
                       n_estimators=100)

In [ ]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(clf, threshold=0.01)
sfm.fit(x,y)

In [ ]:
print(sfm.get_support())
count = -1 
for i in sfm.get_support() :
    count += 1
    if i == True:
        print(count)


In [ ]:
# ##隨機燊齡補植 但已經採用 R 補植好的資料
# # y即目标年龄
# y = df_nonull[:, 0]

# # X即特征属性值
# X = df_nonull[:, 1:]

# # fit到RandomForestRegressor之中
# rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
# rfr.fit(X, y)

#  # 用得到的模型进行未知年龄结果预测
# predictedAges = rfr.predict(unknown_age[:, 1:])
# #     print predictedAges
# # 用得到的预测结果填补原缺失数据
# df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
x, y , test_size=0.2, random_state=0)
y_trainn = np.array(y_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
sc.fit(x)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# coding: utf-8
from sklearn.linear_model import ElasticNet


import numpy as np  
import random   
import matplotlib.pyplot as plt  
max_iter=100 # 跌帶次數
pN = 60  # 人口
clf = ElasticNet(alpha=0.1, l1_ratio=0.7)
#----------------------PSO参数设置---------------------------------  
class PSO():  
    def __init__(self,pN,dim,max_iter):  
        self.w = 0.8    
        self.c1 = 1.96     
        self.c2 = 1.96     
        self.r1= 0.6  
        self.r2=0.3
        self.Vbest = 0.3             # 最高速度
        self.pN = pN                #粒子数量  
        self.dim = dim              #搜索维度  
        self.max_iter = max_iter    #迭代次数  
        self.X = np.zeros((self.pN,self.dim))       #所有粒子的位置和速度  
        self.V = np.zeros((self.pN,self.dim))  
        self.pbest = np.zeros((self.pN,self.dim))   #個體經歷的最佳位置和全局最佳位置  
        self.gbest = np.zeros((1,self.dim))  
        self.p_fit = np.zeros(self.pN)              #每個個體的歷史最佳值  
        self.fit = 1e10             #全局最佳值  
          
#---------------------目标函数Sphere函数-----------------------------  
    def function(self,x): 
        clf = ElasticNet(alpha=x[0] , l1_ratio=x[1])
        clf.fit(X_train_std,y_train)
        pred = clf.predict(X_train_std)
        n = X_train.shape[0] 
        rmse = 0.0
        for i in range(n):
            rmse += (pred[i] - y_trainn[i]) ** 2
        rmse = (rmse/n) ** 0.5
        return rmse
#---------------------初始化种群----------------------------------  
    def init_Population(self):  
        for i in range(self.pN):  
            for j in range(self.dim):  
                self.X[i][j] = random.uniform(0,1)  
                self.V[i][j] = random.uniform(0,1)  
            self.pbest[i] = self.X[i]  
            tmp = self.function(self.X[i])  
            self.p_fit[i] = tmp  
            if(tmp < self.fit):  
                self.fit = tmp  
                self.gbest = self.X[i]  
      
#----------------------更新粒子位置----------------------------------  
    def iterator(self):  
        fitness = []  
        for t in range(self.max_iter):  
            for i in range(self.pN):         #更新gbest\pbest  
               temp = self.function(self.X[i])  
               if(temp<self.p_fit[i]):      #更新个体最优  
                   self.p_fit[i] = temp  
                   self.pbest[i] = self.X[i]  
                   if(self.p_fit[i] < self.fit):  #更新全局最优  
                       self.gbest = self.X[i]  
                       self.fit = self.p_fit[i]  
            for i in range(self.pN): 
                temp = self.w*self.V[i] + self.c1*self.r1*(self.pbest[i] - self.X[i]) + \
                            self.c2*self.r2*(self.gbest - self.X[i])  
                for z in range(self.dim):
                    if temp[z]  >= self.Vbest:
                        temp[z] = self.Vbest
                self.V[i] = temp
                self.X[i] = self.X[i] + self.V[i]  
            fitness.append(self.fit)  
            print("第%d次" %  (t + 1) ,self.fit,self.gbest)                   #输出最优值  
        return fitness  
 
#----------------------程序执行-----------------------  
my_pso = PSO(pN=pN,dim=2,max_iter=max_iter)  
my_pso.init_Population()  
fitness = my_pso.iterator()
#-------------------画图--------------------  
plt.figure(1)  
plt.title("Figure1")  
plt.xlabel("iterators", size=14)  
plt.ylabel("fitness", size=14)  
t = np.array([t for t in range(0,max_iter)])  
fitness = np.array(fitness)  
plt.plot(t,fitness, color='b',linewidth=3)  
plt.show()

In [ ]:
clf =  ElasticNet(alpha=0.06095951 , l1_ratio=1.2)
clf.fit(X_train_std,y_train)
pred = clf.predict(X_test_std)
n = X_test_std.shape[0] 
rmse = 0.0
y_testt = np.array(y_test)
for i in range(n):
    rmse += (pred[i] - y_testt[i]) ** 2
rmse = (rmse/n) ** 0.5
print("RMSE" , rmse)
    